<a href="https://colab.research.google.com/github/muruganveera/murugan/blob/master/Copy_of_LSTM_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
from google.colab import drive
drive.mount('/content/drive')
B0005=loadmat('/content/drive/MyDrive/B0005.mat')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
  import os
  drive.mount('/content/drive')

In [ ]:
pip install scipy

In [ ]:
import numpy as np
"from mat2json import loadMat"
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import LeakyReLU
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
def extract_discharge(Battery):

    cap = []
    i = 1
    for Bat in Battery:
        if Bat['cycle'] == 'discharge':
            cap.append((Bat['data']['Capacity'][0]))
            i+=1
    return cap

In [ ]:
def extract_charge_VIT(Battery):

    cap = []
    i = 1
    for Bat in Battery:
        if Bat['cycle'] == 'charge':
            cap.append((Bat['data']['Capacity'][0]))
            i+=1
    return cap

In [ ]:
A = extract_charge_VIT(B0005)
#print(A)
InitC = 1.85;
cap5 = extract_discharge(B0005);

In [ ]:
xData_raw = A
comp = len(A) - len(cap5);
yData_raw = np.vstack((InitC*np.ones((comp, 1)), np.reshape(cap5, (len(cap5), 1))))

NameError: ignored

In [ ]:
xminmax = MinMaxScaler(feature_range=(0, 1)) # xData_raw feature scaling
xData = xminmax.fit_transform(xData_raw) # feature scaling of xData_raw
#print(xData)
yminmax = MinMaxScaler(feature_range=(0, 1)) # yData_raw feature scaling
yData = yminmax.fit_transform(yData_raw) # feature scaling of yData_raw
#print(yData)

In [ ]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(xData, yData, test_size = 0.20,shuffle = False)# split the data into train and test

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

# Training Phase
model.summary()

In [ ]:
#Predicting network output
y_predict = model.predict(X_test) # predictions on test data

In [ ]:
# calculate the prediction error
mape = np.sum(abs(y_test[:,0]- y_predict))/np.size(y_test)
print(mape)

In [ ]:
y_predict_actual = yminmax.inverse_transform(y_predict.reshape(-1,1)) # get the original a

In [ ]:
y_test_actual = yminmax.inverse_transform(y_test[:,0].reshape(-1,1)) # get the original

In [ ]:
plt.plot(y_predict_actual)

In [ ]:
plt.plot(y_test_actual)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.plot(np.arange(57), 1.4*np.ones((57, 1)),'k--',linewidth = 2)
ax.plot(y_predict_actual, color='black',label='Predicted Capacity')
ax.plot(y_test_actual, color='red',label='Actual Capacity')
ax.set(xlabel='Discharge Cycles', ylabel='Capacity(Ah)')
ax.set_xlim([0,57])
#ax.set_ylim([0,2])
ax.legend()
ax.set(title = 'LSTM')

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();